# 📘 Descrição do Notebook

Este notebook realiza a **extração, tratamento, armazenamento e validação de dados** da **Pokémon API**, com foco nos Pokémons **Charmander**, **Charmeleon** e **Charizard**.

Agora também integra as **habilidades** de cada Pokémon e implementa um sistema de alerta avançado, inspirado no modelo da Aula 20.

---

## 🔹 Primeiro passo: Mensagem de Alerta
- Implementação da função `alerta()` para gerar mensagens personalizadas em caso de erro de requisição ou eventos específicos durante a execução do notebook.

## 🔹 Etapa 1: Extração dos Dados
- Utilização da biblioteca `requests` para acessar a API pública da PokéAPI;
- Extração dos dados principais dos Pokémons, como Nome, Experiência Base, Altura, Peso e ID.

## 🔹 Etapa 2: Coleta e Organização
- Coleta dos dados dos Pokémons selecionados (Charmander, Charmeleon e Charizard);
- Organização dos dados coletados em um `DataFrame` do pandas.

## 🔹 Etapa 3: Tratamento dos Dados
- Padronização dos nomes das colunas com `str.title()`;
- Preenchimento de valores ausentes com `fillna(0)`;
- Conversão dos tipos de dados (altura e peso para `float`);
- Padronização dos nomes dos Pokémons.

## 🔹 Etapa 4: Armazenamento no SQLite
- Utilização do `sqlite3` para criar e conectar a um banco de dados local (`pokemons_v4.db`);
- Salvamento dos dados tratados na tabela `tabela_pokemons`.

## 🔹 Etapa 5: Validação
- Leitura dos dados diretamente do banco de dados SQLite;
- Exibição da tabela recuperada para conferência com `display()`.

## 🔹 Etapa 6: Sistema de Alerta
- Verificação automática de Pokémons com **experiência base acima de 200**;
- Geração de alertas personalizados destacando o nome do Pokémon e seu valor de experiência.

---

> 💡 Este notebook demonstra, na prática, como integrar múltiplas APIs, tratar dados com `pandas`, salvar informações em banco de dados local com `sqlite3` e implementar um sistema de alerta automático para eventos específicos.




In [1]:
import requests
import pandas as pd
import sqlite3

def alerta(mensagem):
    print(f"🚨 Alerta: {mensagem}")

## Etapa 1: Extração dos Dados

In [2]:
def extrair_dados_pokemon(id_pokemon):
    url = f'https://pokeapi.co/api/v2/pokemon/{id_pokemon}'
    res = requests.get(url)
    if res.status_code != 200:
        alerta(f"Erro ao acessar Pokémon ID {id_pokemon}")
        return None
    data = res.json()
    return {
        'ID': data['id'],
        'Nome': data['name'].title(),
        'Experiência Base': data['base_experience'],
        'Altura': data['height'],
        'Peso': data['weight'],
        'Habilidades': ', '.join([h['ability']['name'] for h in data['abilities']])
    }

## Etapa 2: Coleta e Organização

In [3]:
pokemons = [4, 5, 6]  # Charmander, Charmeleon, Charizard
dados = []
for pid in pokemons:
    d = extrair_dados_pokemon(pid)
    if d:
        dados.append(d)
df_pokemons = pd.DataFrame(dados)
display(df_pokemons)

,ID,Nome,Experiência Base,Altura,Peso,Habilidades
0,4,Charmander,62,6,85,"blaze, solar-power"
1,5,Charmeleon,142,11,190,"blaze, solar-power"
2,6,Charizard,267,17,905,"blaze, solar-power"


## Etapa 3: Tratamento dos Dados

In [4]:
df_pokemons.columns = df_pokemons.columns.str.title()
df_pokemons = df_pokemons.fillna(0)
df_pokemons['Altura'] = df_pokemons['Altura'].astype(float)
df_pokemons['Peso'] = df_pokemons['Peso'].astype(float)
display(df_pokemons)

,Id,Nome,Experiência Base,Altura,Peso,Habilidades
0,4,Charmander,62,6.0,85.0,"blaze, solar-power"
1,5,Charmeleon,142,11.0,190.0,"blaze, solar-power"
2,6,Charizard,267,17.0,905.0,"blaze, solar-power"


## Etapa 4: Armazenamento no SQLite

In [5]:
with sqlite3.connect('pokemons_v4.db') as conn:
    df_pokemons.to_sql('tabela_pokemons', conn, if_exists='replace', index=False)
print('Dados salvos no banco pokemons_v4.db')

Dados salvos no banco pokemons_v4.db


## Etapa 5: Validação

In [6]:
with sqlite3.connect('pokemons_v4.db') as conn:
    df_val = pd.read_sql('SELECT * FROM tabela_pokemons', conn)
display(df_val)

,Id,Nome,Experiência Base,Altura,Peso,Habilidades
0,4,Charmander,62,6.0,85.0,"blaze, solar-power"
1,5,Charmeleon,142,11.0,190.0,"blaze, solar-power"
2,6,Charizard,267,17.0,905.0,"blaze, solar-power"


## Etapa 6: Sistema de Alerta

In [7]:
for _, row in df_pokemons.iterrows():
    if row['Experiência Base'] > 200:
        alerta(f"{row['Nome']} possui experiência base elevada: {row['Experiência Base']}")

🚨 Alerta: Charizard possui experiência base elevada: 267


### 🔍 Conclusão

Este notebook demonstrou um processo completo de extração, transformação, armazenamento e validação de dados, utilizando a API oficial PokéAPI para os Pokémons **Charmander**, **Charmeleon** e **Charizard**.

As principais etapas realizadas foram:

- Definição de uma função `alerta()` para gerar mensagens personalizadas em caso de erro ou eventos importantes.
- Acesso à API com `requests` para obter dados brutos dos Pokémons, incluindo suas habilidades.
- Organização dos dados principais em tabelas com `pandas`, incorporando nome, experiência base, altura, peso, ID e habilidades.
- Tratamento dos dados, incluindo padronização de nomes, tipos de variáveis (`float`) e preenchimento de valores ausentes.
- Armazenamento dos dados tratados em um banco de dados SQLite (`pokemons_v4.db`) na tabela `tabela_pokemons`.
- Validação dos dados armazenados ao ler diretamente do banco de dados e exibir os resultados.
- Implementação de um sistema de alerta avançado, que detecta automaticamente Pokémons com mais de 200 de experiência base e exibe mensagens de notificação.

Este projeto serviu como prática de integração de múltiplas fontes de dados, manipulação com `pandas`, persistência em banco de dados com `sqlite3` e criação de sistemas automáticos de alerta, consolidando conceitos importantes para projetos de dados no mundo real.


